In [1]:
import zipfile
import os

# Unzip using zipfile
with zipfile.ZipFile('fundamental-research-project.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


In [2]:
import shutil

shutil.rmtree('fundamental-research-project')

In [ ]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd
from typing import List
from tabulate import tabulate
from dataclasses import dataclass, field

# Trainer
from src.trainer.trainer import Trainer

# Models
from src.models.chA_p4_cnn import A_Ch_P4CNN
from src.models.spA_p4_cnn import A_Sp_P4CNN
from src.models.fA_p4_allcnn import fA_P4AllCNNC
from src.models.p4_allcnn import P4AllCNNC

# Data Utils
from src.datasets.rot_mnist_dataset import get_dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
 

AttributeError: _ARRAY_API not found

## rot-MNIST dataset

In [2]:

train_loader, val_loader, test_loader = get_dataset(batch_size=128, num_workers=2)



In [13]:
img, label = full_train[0]
print(img.shape)

torch.Size([1, 28, 28])


## Experiment

In [6]:
# ----- Helper Functions -----
def init_model(name):
    if name == "chA_p4_cnn":
        return A_Ch_P4CNN()
    elif name == "spA_p4_cnn":
        return A_Sp_P4CNN()
    elif name == "fA_p4_cnn":
        return fA_P4AllCNNC
    elif name == "p4_cnn":
        return P4AllCNNC
    else:
        raise ValueError(f"Unknown model name: {name}")

def init_optimizer(model, lr, weight_decay):
    return optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

def init_scheduler(optimizer, milestones):
    return optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 250, 300], gamma=0.1)

# ----- Helper Classes -----
@dataclass
class HyperParams:
    lr: float
    epochs: int
    weight_decay: float
    momentum: float
    gamma: float
    milestones: List[int] = field(default_factory=list)

In [7]:
# ----- Configuration -----
num_iterations = 1
log_dir = "../logs"

model_hyperparameters = {
    "chA_p4_cnn":  HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    "spA_p4_cnn": HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    "fA_p4_cnn": HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    "p4_cnn":  HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1)
}
model_names = model_hyperparameters.keys()
accuracies = {name: [] for name in model_names}

In [8]:
# ----- Main Training Loop -----
for it in range(num_iterations):
    print(f"Iteration {it + 1}/{num_iterations}")

    for name in model_names:
        print(f"\n→ Training model: {name}")

        # 1. Grab hyperparams for this model
        hp = model_hyperparameters[name]

        # 2. Initialize model, criterion, optimizer, scheduler
        model = init_model(name)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(
            model.parameters(),
            lr=hp.lr,
            momentum=hp.momentum,
            weight_decay=hp.weight_decay
        )
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=hp.milestones,
            gamma=hp.gamma
        )

        # 3. Wrap in our Trainer (single‐model)
        trainer = Trainer(
            models={name: model},
            optimizers=[optimizer],
            criterions=[criterion],
            schedulers=[scheduler],
            log_dir=f"{log_dir}/{name}"
        )

        # 4. Train & validate with the model‐specific epoch count
        trainer.train(
            num_epochs=hp.epochs,
            train_loader=train_loader,
            val_loader=val_loader,
        )

        # 5. Evaluate on test set and record accuracy
        test_acc = trainer.evaluate(test_loader=test_loader)[name]
        accuracies[name].append(test_acc)

Iteration 1/1

→ Training model: chA_p4_cnn
Using device: cpu
Training model : chA_p4_cnn


Training Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


ValueError: _set_worker_pids should be called only once for each _BaseDataLoaderIter.

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "<string>", line 1, in <module>
  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
  File "/Users/larabastos/miniconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
        self = reduction.pickle.load(from_parent)self = reduction.pickle.load(from_parent)

                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/torch/multiprocessing/reductions.py",

In [9]:
# ----- Final Statistics -----
final_stats = {
    name: {
        "% Test error": (1 - float(np.mean(vals))) * 100,
        "% std": float(np.std(vals)) * 100,
        "Num Parameters": sum(p.numel() for p in init_model(name).parameters())
    }
    for name, vals in accuracies.items()
}

## Table generation

In [10]:
df = pd.DataFrame.from_dict(final_stats, orient='index')
df = df.round(2)
print("📊 Model Accuracy Summary in CIFAR10\n")
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

📊 Model Accuracy Summary in CIFAR10

╒══════════════╤════════════════╤═════════╤══════════════════╕
│              │   % Test error │   % std │   Num Parameters │
╞══════════════╪════════════════╪═════════╪══════════════════╡
│ p4_allcnn    │          54.47 │       0 │      1.37037e+06 │
├──────────────┼────────────────┼─────────┼──────────────────┤
│ fA_p4_allcnn │          62.3  │       0 │      1.40012e+06 │
├──────────────┼────────────────┼─────────┼──────────────────┤
│ p4m_resnet   │          53.85 │       0 │      2.62168e+06 │
╘══════════════╧════════════════╧═════════╧══════════════════╛
